In [1]:
import ABCD_ML

In [2]:
nda_dr = '/mnt/sdb2/2.0_ABCD_Data_Explorer/2.0_NDA_Data/'
test_mapping_loc = nda_dr + 'ABCD_Release_ Notes_Data_Release_ 2.0/22. ABCD_Release_2.0_mapping_r.csv'
test_data_loc = nda_dr + 'MRI/ABCD sMRI Part 1.csv'
test_target_loc = nda_dr + 'Mental Health/ABCD Parent Demographics Survey.csv'
test_exclusion_loc = '/home/sage/bader_things/invalid_pguids.txt'

In [3]:
ML = ABCD_ML.ABCD_ML()

ML.load_name_map(loc = test_mapping_loc,
                 source_name_col = "NDAR name",
                 target_name_col = "REDCap name/NDA alias")

ML.load_data(loc=test_data_loc,
             dataset_type='explorer',
             filter_outlier_percent=.0005, 
             winsorize_val=.001)

ML.load_targets(loc=test_target_loc,
                col_name='demo_relig_v2',
                data_type='c')
ML.load_exclusions(loc=test_exclusion_loc)

ABCD_ML object initialized
Loaded map file
Loading /mnt/sdb2/2.0_ABCD_Data_Explorer/2.0_NDA_Data/MRI/ABCD sMRI Part 1.csv assumed to be dataset type: explorer
dropped ['abcd_smrip101_id', 'dataset_id', 'smri_visitid'] columns by default due to dataset type
Dropped 0 columns, per drop_keys argument
Dropped 0 cols for all missing values
Dropped 522 rows for missing values
Dropped rows with missing data
Filtered data for outliers with value:  0.0005
Winsorized data with value:  0.001
loaded shape:  (8725, 749)

Total valid (overlapping subjects / not in exclusions) subjects = 8725

Loading  /mnt/sdb2/2.0_ABCD_Data_Explorer/2.0_NDA_Data/Mental Health/ABCD Parent Demographics Survey.csv
Encoded to 17 categories
Final shape:  (11310, 17)

Total valid (overlapping subjects / not in exclusions) subjects = 8336

Total excluded subjects:  1137

Total valid (overlapping subjects / not in exclusions) subjects = 7547

Removing non overlapping + excluded subjects


In [4]:
ML.define_validation_strategy(stratify=ML.original_targets_key)

CV defined with stratifying behavior, over 17 unique values.


We are defining our validation strategy as stratifying over the target keys, as some of the religions have very little representation, and we could potentially throw errors if some value is not represented in one of the various validation folds - We still might if we use too much internal CV.

In [5]:
ML.train_test_split(test_size=.2,
                    random_state=9)

Final data for modeling loaded shape: (7547, 767)
Performed train/test split, train size: 6037 test size:  1510


In [6]:
ML.set_default_ML_params(problem_type = 'categorical',
                         data_scaler = 'standard',
                         n_splits = 2,
                         n_repeats = 1,
                         int_cv = 2,
                         class_weight = 'balanced',
                         n_jobs = 4,
                         n_iter = 2,
                         random_state = 9)

No default metric passed, set to, weighted roc auc based on default problem type.
No default extra params passed, set to empty dict
Default params set.



In [7]:
ML.show_model_types(problem_type='categorical')

Note: gs and rs are  Grid Search and Random Search
Models with gs or rs will have their hyper-parameters tuned accordingly.

Problem Type: categorical
----------------------
Avaliable models: 

Model str indicator:  dt classifier
(MultiClass)
Model object:  <class 'sklearn.tree.tree.DecisionTreeClassifier'>

Model str indicator:  dt classifier gs
(MultiClass)
Model object:  <class 'sklearn.model_selection._search.GridSearchCV'>

Model str indicator:  gaussian nb
(MultiClass)
Model object:  <class 'sklearn.naive_bayes.GaussianNB'>

Model str indicator:  knn classifier
(MultiClass)
Model object:  <class 'sklearn.neighbors.classification.KNeighborsClassifier'>

Model str indicator:  knn classifier gs
(MultiClass)
Model object:  <class 'sklearn.model_selection._search.GridSearchCV'>

Model str indicator:  light gbm classifier
(MultiClass)
Model object:  <class 'lightgbm.sklearn.LGBMClassifier'>

Model str indicator:  light gbm classifier rs
(MultiClass)
Model object:  <class 'sklearn.model

In [14]:
ML.Evaluate(model_type = 'random forest cal',
            metric = 'macro precision')

Running Evaluate with:
model_type = random forest cal
problem_type = categorical
metric = macro precision
data_scaler = standard
n_splits = 2
n_repeats = 1
int_cv = 2
class_weight = balanced
n_jobs = 4
n_iter = 2
random_state = 9
extra_params = {}

Not all model types passed have multilabel support! Using multiclass instead.


/home/sage/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Macro mean score:  0.37328278899086553
Macro std in score:  0.0
Micro mean score:  0.37328278899086553
Micro std in score:  0.05119126408219368


/home/sage/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


[0.3220915249086719, 0.42447405307305924]

In [15]:
ML.Evaluate(model_type = 'random forest cal',
            metric = 'macro roc auc')

Running Evaluate with:
model_type = random forest cal
problem_type = categorical
metric = macro roc auc
data_scaler = standard
n_splits = 2
n_repeats = 1
int_cv = 2
class_weight = balanced
n_jobs = 4
n_iter = 2
random_state = 9
extra_params = {}

Not all model types passed have multilabel support! Using multiclass instead.
Macro mean score:  0.8204773375764192
Macro std in score:  0.0
Micro mean score:  0.8204773375764192
Micro std in score:  0.008550866658819045


[0.8119264709176002, 0.8290282042352383]

In [ ]:
ML.Evaluate(model_type = 'logistic cv',
            metric = 'macro roc auc')

Running Evaluate with:
model_type = logistic cv
problem_type = categorical
metric = macro roc auc
data_scaler = standard
n_splits = 2
n_repeats = 1
int_cv = 2
class_weight = balanced
n_jobs = 4
n_iter = 2
random_state = 9
extra_params = {}

Not all model types passed have multilabel support! Using multiclass instead.


In [10]:
#ML.Evaluate(model_type = 'knn classifier',
#           metric = 'hamming'
#          )

In [11]:
#ML.Evaluate(model_type = 'knn classifier gs',
#           metric = 'hamming'
#          )

In [12]:
#ML.Evaluate(model_type = 'rf',
#            metric = 'micro f1')